# Get the dataset

In [1]:
from typing import Tuple
import numpy as np
import config

dataset_image = np.load(config.x_path, mmap_mode='c')[0:4096]
dataset_label = np.load(config.y_path, mmap_mode='c')[0:4096]
assert(dataset_image.shape[0] == dataset_label.shape[0])
print(f'''Stats:
    | Number of punching: {dataset_label.sum()}
    | Number of not-punching: {dataset_label.shape[0] - dataset_label.sum()}''')


Stats:
    | Number of punching: 206
    | Number of not-punching: 3890


# Boilerplate Code

In [2]:
from sklearn.model_selection import RepeatedKFold
import config

kfold = RepeatedKFold(
    n_splits=config.kfold_nsplits,
    n_repeats=config.kfold_nrepeats,
    random_state=config.seed
)


In [3]:
import torch
class Data(torch.utils.data.Dataset):
    def __init__(self, image: np.ndarray, label: np.ndarray, indices: np.ndarray) -> None:
        assert(image.shape[0] == label.shape[0])
        self.image = image
        self.label = label
        self.indices = indices
    
    def __len__(self) -> int:
        return self.indices.shape[0]
    
    def __getitem__(self, idx: int) -> 'Tuple[np.ndarray, bool]':
        idx = self.indices[idx]
        return self.image[idx], self.label[idx]


# Model

In [4]:
from torch import nn


In [5]:
def print_model(model: nn.Module):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
    
    param_size  = param_size / 1024**2
    buffer_size = buffer_size / 1024**2
    print(f'Param size: {param_size:.3f}MB')
    print(f'Buffer size: {buffer_size:.3f}MB')

def print_tensor(tensor: torch.Tensor):
    size_gb = tensor.element_size() * tensor.nelement() / (1<<30)
    print(f"{size_gb:.3f}GB")



In [6]:
model = config.get_model()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

model = model.to(config.device)
criterion = criterion.to(config.device)


# Train

In [7]:
from tqdm import tqdm
def train(train_idx: np.ndarray) -> 'Tuple(float, float)':
    train = Data(dataset_image, dataset_label, train_idx)
    train_dataloader = torch.utils.data.DataLoader(
        train, batch_size=config.batch_size)
    total_loss_train = 0
    total_accumulate_train = 0
    model.train()
    for image, label in tqdm(train_dataloader):
        image = image.to(config.device, dtype=torch.float)
        label = label.to(config.device, dtype=torch.uint8)
        image = config.random_transforms(image)
        output = model(image)

        batch_loss = criterion(output, label)

        if np.isnan(batch_loss.detach().cpu().numpy()).sum():
            print(batch_loss, total_loss_train, label)
            return
            
        total_loss_train += batch_loss.item()

        accumulate = (output.argmax(dim=1) == label).sum()
        total_accumulate_train += accumulate.item()

        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        model.zero_grad()

    print(total_loss_train, total_accumulate_train)

    total_loss_train = total_loss_train.item()
    total_accumulate_train = total_accumulate_train.item()
    
    return (total_loss_train, total_accumulate_train)


In [8]:
def judge(judge_idx: np.ndarray) -> 'Tuple(float, float)':
    judge = Data(dataset_image, dataset_label, judge_idx)
    judge_dataloader = torch.utils.data.DataLoader(
        judge, batch_size=config.batch_size)

    total_loss_judge = 0
    total_accumulate_judge = 0
    with torch.no_grad():
        for image, label in tqdm(judge_dataloader):
            image = image.to(config.device, dtype=torch.float)
            label = label.to(config.device, dtype=torch.uint8)

            output = model(image)
            batch_loss = criterion(output, label)
            total_loss_judge += batch_loss.item()

            accumulate = (output.argmax(dim=1) == label).sum()
            total_accumulate_judge += accumulate.item()

    total_loss_judge = total_loss_judge.item()
    total_accumulate_judge = total_accumulate_judge.item()
    return total_loss_judge, total_accumulate_judge

In [9]:
min_judge_loss = float('inf')


for epoch, (train_idx, judge_idx) in enumerate(kfold.split(dataset_label)):
    print(f'''Starting epoch {epoch+1}
        | Train size: {train_idx.shape[0]}
        | Judge size: {judge_idx.shape[0]}''')
    total_loss_train, total_accumulate_train = train(train_idx)
    total_loss_judge, total_accumulate_judge = judge(judge_idx)

    print(
        f'''Epoch: {epoch+1} 
        | Train Loss: {total_loss_train / len(train_idx):.3f}
        | Train Accuracy: {total_accumulate_train/len(train_idx):.3f}
        | Val Loss: {total_loss_judge/len(judge_idx):.3f}
        | Val Accuracy: {total_accumulate_judge/len(judge_idx):.3f}'''
        
    )
    if min_judge_loss > total_loss_judge/len(judge_idx):
        min_judge_loss = total_loss_judge/len(judge_idx)
        torch.save(model.state_dict(), config.model_path)
        print(f"Save model because val loss improve loss {min_judge_loss:.3f}")

Starting epoch 1
        | Train size: 3686
        | Judge size: 410


  1%|          | 25/3686 [00:03<07:29,  8.14it/s] 


tensor(nan, device='cuda:0', grad_fn=<NllLossBackward0>) 15.21963266655802 tensor([0], device='cuda:0', dtype=torch.uint8)


TypeError: cannot unpack non-iterable NoneType object